In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image

device = "cuda:0"

modelPath = "nikravan/glm-4vq"
tokenizer = AutoTokenizer.from_pretrained(modelPath, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    modelPath,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto",
    use_cache=True,
)

/home/boom/.pyenv/versions/aivet/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/boom/.pyenv/versions/aivet/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

/home/boom/.pyenv/versions/aivet/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
image = Image.open("../data/input/202410_hope_bio.pdf").convert("RGB")

UnidentifiedImageError: cannot identify image file '/home/boom/sdev/repos/aivet/data/input/202410_hope_bio.pdf'

In [ ]:
import glob, sys, fitz

# To get better resolution
zoom_x = 2.0  # horizontal zoom
zoom_y = 2.0  # vertical zoom
mat = fitz.Matrix(zoom_x, zoom_y)  # zoom factor 2 in each dimension


for filename in ["../data/input/202410_hope_bio.pdf"]:
    doc = fitz.open(filename)  # open document
    for page in doc:  # iterate through the pages
        pix = page.get_pixmap(matrix=mat)  # render page to an image
        pix.save("../data/out/page-%i.png" % page.number)

In [2]:
image = Image.open("data/input/Analisi Hope pagina 1.jpg").convert("RGB")

In [3]:
query = """Task: Extract and list all blood test details.

Objective:
1. Identify and extract **chemical names** from the blood test report.
2. Capture the **detected levels** of each chemical.
3. Extract the **normal range** for each corresponding chemical.

Expected Output Format:
- Chemical Name: [Name]
  - Detected Level: [Value]
  - Normal Range: [Min Value] - [Max Value]

Please ensure accurate extraction, including any unit symbols (e.g., mg/dL, IU/L), and handle variations in formatting or alignment. Return results in a structured list format for easy readability."""
# image=""
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "image": image, "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)  # chat with image mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature": 0.2}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1] :]
    print(tokenizer.decode(outputs[0]))

2024-11-11 11:29:47.664521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731320987.732291    7898 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731320987.754947    7898 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 11:29:47.903401: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Certainly, here are the extracted blood test details from the provided image:

- Chemical Name: RBC (Red Blood Cells)
  - Detected Level: 8.51 MjL
  - Normal Range: 5.65 - 8.87 MjL
- Chemical Name: HCT (Hematocrit)
  - Detected Level: 56.7%
  - Normal Range: 37.3 - 61.7%
- Chemical Name: HGB (Hemoglobin)
  - Detected Level: 19.5 g/dL
  - Normal Range: 13.1 - 20.5 g/dL
- Chemical Name: MCV (Mean Corpuscular Volume)
  - Detected Level: 66.6 fL
  - Normal Range: 61.6 - 73.5 fL
- Chemical Name: MCH (Mean Corpuscular Hemoglobin)
  - Detected Level: 22.9 pg
  - Normal Range: 21.2 - 25.9 pg
- Chemical Name: MCHC (Mean Corpuscular Hemoglobin Concentration)
  - Detected Level: 34.4 g/dL
  - Normal Range: 32.0 - 37.9 g/dL
- Chemical Name: RDW (Red Cell Distribution Width)
  - Detected Level: 19.0%
  - Normal Range: 13.6 - 21.7%
- Chemical Name: %RETIC (Reticulocytes)
  - Detected Level: 0.6%
  - Normal Range: 0.5 - 1.5%
- Chemical Name: RETIC (Reticulocytes)
  - Detected Level: 47.7 KjL
  - Norm

In [4]:
query = "Extract all the blood test chemical name values, their detected level and the corresponding normal range"
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "image": image, "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)  # chat with image mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature": 0.2}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1] :]
    print(tokenizer.decode(outputs[0]))

Certainly, here are the blood test chemical name values, their detected levels, and the corresponding normal ranges for the dog named Hope (CORBI):

1. RBC (Red Blood Cells): 8.51 M/uL
   - Detected Level: 8.51 M/uL
   - Normal Range: 5.65 - 8.87 M/uL

2. HCT (Hematocrit): 56.7%
   - Detected Level: 56.7%
   - Normal Range: 37.3 - 61.7%

3. HGB (Hemoglobin): 19.5 g/dL
   - Detected Level: 19.5 g/dL
   - Normal Range: 13.1 - 20.5 g/dL

4. MCV (Mean Corpuscular Volume): 66.6 fL
   - Detected Level: 66.6 fL
   - Normal Range: 61.6 - 73.5 fL

5. MCH (Mean Corpuscular Hemoglobin): 22.9 pg
   - Detected Level: 22.9 pg
   - Normal Range: 21.2 - 25.9 pg

6. MCHC (Mean Corpuscular Hemoglobin Concentration): 34.4 g/dL
   - Detected Level: 34.4 g/dL
   - Normal Range: 32.0 - 37.9 g/dL

7. RDW (Red Cell Distribution Width): 19.0%
   - Detected Level: 19.0%
   - Normal Range: 13.6 - 21.7%

8. %RETIC (Reticulocytes): 0.6%
   - Detected Level: 0.6%
   - Normal Range: 0.5 - 1.6%

9. RETIC (Reticulocyt

In [5]:
query = "Extract all the blood test chemical name values, their detected level and the corresponding normal range only for test values which have abnormal levels."
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "image": image, "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)  # chat with image mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature": 0.2}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1] :]
    print(tokenizer.decode(outputs[0]))

Here are the blood test chemical name values, their detected levels, and the corresponding normal ranges for the tests with abnormal levels:

1. **ProCyte Dx (20 settembre 2021 15:55):**
   - **RBC (RBC):** Detected level: 8.51 MjL, Normal range: 5.65 - 8.87
   - **HCT (Hematocrit):** Detected level: 56.7%, Normal range: 37.3 - 61.7
   - **HGB (Hemoglobin):** Detected level: 19.5 g/dL, Normal range: 13.1 - 20.5
   - **MCV (Mean Corpuscular Volume):** Detected level: 66.6 fL, Normal range: 61.6 - 73.5
   - **MCH (Mean Corpuscular Hemoglobin):** Detected level: 22.9 pg, Normal range: 21.2 - 25.9
   - **MCHC (Mean Corpuscular Hemoglobin Concentration):** Detected level: 34.4 g/dL, Normal range: 32.0 - 37.9
   - **RDW (Red Cell Distribution Width):** Detected level: 19.0%, Normal range: 13.6 - 21.7
   - **%RETIC (Reticulocytes):** Detected level: 0.6%, Normal range: 0.0 - 1.0
   - **RETIC (Reticulocytes):** Detected level: 47.7 KjL, Normal range: 10.0 - 110.0
   - **RETIC-HGB (Reticulocyte

In [7]:
query = "Based on the detected test values in the blood test document as well as the reported reference ranges, identify and list the molecules which are outside the reference ranges. List the molecule name, value and the reference range. Finally perform a health diagnosis based on the abnormalities. Do not list any molecules which are within the reference range, those are considered normal."
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "image": image, "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)  # chat with image mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature": 0.2}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1] :]
    print(tokenizer.decode(outputs[0]))

Based on the blood test document provided, the following molecules are outside the reference ranges:

1. MCHC (Mean Corpuscular Hemoglobin Concentration): The value is 34.4 pg/mL, which is outside the reference range of 21.2 - 25.9 pg/mL.
2. RDW (Red Cell Distribution Width): The value is 19.0%, which is outside the reference range of 13.6 - 21.7%.

Health Diagnosis:

- The elevated MCHC value indicates that the hemoglobin concentration within the red blood cells is higher than normal. This could be due to conditions such as polycythemia vera, where the body produces too many red blood cells, or it could be a secondary effect of dehydration, anemia, or other blood disorders.
- The increased RDW suggests that there is variability in the size of the red blood cells, which can be indicative of anemia, iron deficiency, thalassemia, or other conditions affecting red blood cell production or destruction.

It is important to note that a single blood test result should not be used for a defini